In [ ]:
import pandas as pd
import numpy as np

In [21]:
import os
import cv2
import open3d as o3d
import numpy as np

def project_points(points, K, T):
    """
    Projects 3D points (Nx3) from world (Lidar) coordinates into the 2D image plane.

    Args:
        points (np.ndarray): Nx3 array of 3D points.
        K (np.ndarray): 3x3 camera intrinsic matrix.
        T (np.ndarray): 4x4 extrinsic transformation (world-to-camera).

    Returns:
        proj_points (np.ndarray): Mx2 array of image coordinates.
        valid_mask (np.ndarray): Boolean mask for points with positive depth.
    """
    num_points = points.shape[0]
    # Convert to homogeneous coordinates.
    points_hom = np.hstack((points, np.ones((num_points, 1))))
    
    # Transform points into camera frame.
    points_cam_hom = (T @ points_hom.T).T
    points_cam = points_cam_hom[:, :3]
    
    # Keep only points with positive depth.
    valid_mask = points_cam[:, 2] > 0
    points_cam = points_cam[valid_mask]
    
    # Project onto image plane.
    proj = (K @ points_cam.T).T
    proj[:, 0] /= proj[:, 2]
    proj[:, 1] /= proj[:, 2]
    proj_points = proj[:, :2]
    return proj_points, valid_mask

def overlay_pcd_on_image(image, pcd_path, K, T, point_color=(0, 0, 255), point_radius=5):
    """
    Loads a PCD file, projects its points, and overlays them on the image.
    
    Args:
        image (np.ndarray): Input image.
        pcd_path (str): Path to the .pcd file.
        K (np.ndarray): Camera intrinsic matrix.
        T (np.ndarray): Extrinsic transformation (world-to-camera).
        point_color (tuple): BGR color for points (default red).
        point_radius (int): Circle radius for drawn points.
    
    Returns:
        image (np.ndarray): Image with overlaid points.
    """
    pcd = o3d.io.read_point_cloud(pcd_path)
    points = np.asarray(pcd.points)
    proj_points, _ = project_points(points, K, T)
    
    for pt in proj_points:
        x, y = int(round(pt[0])), int(round(pt[1]))
        # Draw point if within image bounds.
        if 0 <= x < image.shape[1] and 0 <= y < image.shape[0]:
            cv2.circle(image, (x, y), point_radius, point_color, thickness=-1)
    return image

def process_folders(img_folder, pcd_folder, shift, K, T, output_folder, target_size=(720, 480)):
    """
    For each image, finds the corresponding PCD (accounting for a frame shift), overlays the point cloud, and saves the result.
    
    Args:
        img_folder (str): Folder containing images.
        pcd_folder (str): Folder containing PCD files.
        shift (int): Frame shift: image_i corresponds to pcl_(i+shift).
        K (np.ndarray): Camera intrinsic matrix.
        T (np.ndarray): Extrinsic transformation matrix (world-to-camera).
        output_folder (str): Where to save output images.
        target_size (tuple): Resize images to (width, height).
    """
    # List image and PCD files (you may adjust the extensions if needed).
    img_files = sorted([f for f in os.listdir(img_folder) if f.lower().endswith(('.jpg','.png'))])
    pcd_files = sorted([f for f in os.listdir(pcd_folder) if f.lower().endswith('.pcd')])
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for i, img_file in enumerate(img_files):
        pcd_index = i + shift
        if pcd_index < 0 or pcd_index >= len(pcd_files):
            print(f"Skipping {img_file}: corresponding PCD index {pcd_index} is out of range.")
            continue
        
        img_path = os.path.join(img_folder, img_file)
        pcd_path = os.path.join(pcd_folder, pcd_files[pcd_index])
        
        image = cv2.imread(img_path)
        if image is None:
            print(f"Error reading image {img_path}. Skipping.")
            continue

        # Resize to expected dimensions.
        image = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
        
        # Overlay Lidar points.
        overlaid_image = overlay_pcd_on_image(image, pcd_path, K, T)
        
        output_path = os.path.join(output_folder, img_file)
        cv2.imwrite(output_path, overlaid_image)
        print(f"Processed {img_file} with {pcd_files[pcd_index]} -> saved to {output_path}")

if __name__ == "__main__":
    # ----------------------------
    # Camera Intrinsic Parameters
    # ----------------------------
    # From your camera settings:
    #   ImageWidth = 720, ImageHeight = 480, FOV = 90°
    image_width, image_height = 720, 480
    fov_angle = 90
    # Compute focal length: f = (width/2) / tan(FOV/2)
    focal_length = image_width / (2 * np.tan(np.radians(fov_angle) / 2))  # should be 360
    cx, cy = image_width / 2, image_height / 2
    K = np.array([[focal_length, 0, cx],
                  [0, focal_length, cy],
                  [0, 0, 1]])
    print("Camera intrinsic matrix K:")
    print(K)
    
    # -----------------------------------------------------
    # Define the (working) transformation from world to camera.
    # This mimics the working script:
    #   1. Subtract the camera's world position.
    #   2. Rotate with R.
    # Where:
    #   Camera position (in world coordinates): [300.001, 0.0, 260.0]
    #   Rotation (world -> camera): 
    #       X_world -> Z_cam
    #       Y_world -> X_cam
    #       Z_world -> -Y_cam
    # -----------------------------------------------------
    t_cam = np.array([300.001, 0.0, 260.0])
    R_wc = np.array([
        [ 0, -1,  0],  # sim Y -> -X (camera)
        [ 0,  0,  -1],  # sim Z ->  Y  (camera)
        [ 1,  0,  0]   # sim X ->  Z  (camera)
    ])
    # Build the extrinsic matrix T = [R_wc, -R_wc * t_cam]
    T_world_to_cam = np.eye(4)
    T_world_to_cam[:3, :3] = R_wc
    T_world_to_cam[:3, 3] = -R_wc @ t_cam
    print("Extrinsic transformation matrix T (world-to-camera):")
    print(T_world_to_cam)
    
    # ----------------------------
    # Directories and Shift Setting
    # ----------------------------
    # (Update these paths as needed.)
    img_folder = "./imgs1"
    pcd_folder = "./pcl"
    output_folder = "./output"
    
    # Set the shift parameter.
    # For example, if shift = 57 then image_0 corresponds to pcl57.pcd.
    shift = -47
    
    process_folders(img_folder, pcd_folder, shift, K, T_world_to_cam, output_folder, target_size=(image_width, image_height))


Camera intrinsic matrix K:
[[360.   0. 360.]
 [  0. 360. 240.]
 [  0.   0.   1.]]
Extrinsic transformation matrix T (world-to-camera):
[[   0.      -1.       0.       0.   ]
 [   0.       0.      -1.     260.   ]
 [   1.       0.       0.    -300.001]
 [   0.       0.       0.       1.   ]]
Skipping image_1.jpg: corresponding PCD index -47 is out of range.
Skipping image_10.jpg: corresponding PCD index -46 is out of range.
Skipping image_100.jpg: corresponding PCD index -45 is out of range.
Skipping image_101.jpg: corresponding PCD index -44 is out of range.
Skipping image_102.jpg: corresponding PCD index -43 is out of range.
Skipping image_103.jpg: corresponding PCD index -42 is out of range.
Skipping image_104.jpg: corresponding PCD index -41 is out of range.
Skipping image_105.jpg: corresponding PCD index -40 is out of range.
Skipping image_106.jpg: corresponding PCD index -39 is out of range.
Skipping image_107.jpg: corresponding PCD index -38 is out of range.
Skipping image_108.jp